In [73]:
import cv2 
import numpy as np
import imutils

### Carregando a Imagem

In [74]:
image = cv2.imread('teste4.jpeg')
image = cv2.resize(image, (image.shape[1]//2,image.shape[0]//2))
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Identificando bordas

In [75]:
gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

grayBlur = cv2.blur(gray.copy(), (3, 3))

edgedImage = cv2.Canny(grayBlur.copy(), 50, 450, 3)

cv2.imshow('edgedImage', edgedImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Detectando as bordas

In [76]:
allContours = cv2.findContours(edgedImage.copy(),
                               cv2.RETR_LIST, 
                               cv2.CHAIN_APPROX_SIMPLE)
allContours = imutils.grab_contours(allContours)

allContours = sorted(allContours, key = cv2.contourArea, reverse = True)[:1]

perimeter = cv2.arcLength(allContours[0], True)
ROIdimensions = cv2.approxPolyDP(allContours[0], 0.02*perimeter, True)

In [77]:
ROIdimensions = ROIdimensions.reshape(4,2)
rect = np.zeros((4,2), dtype= "float32" )

s = np.sum(ROIdimensions, axis=1)
rect[0] = ROIdimensions[np.argmin(s)]
rect[2] = ROIdimensions[np.argmax(s)]

diff = np.diff(ROIdimensions, axis=1)
rect[1] = ROIdimensions[np.argmin(diff)]
rect[3] = ROIdimensions[np.argmax(diff)]

(tl, tr, br, bl) = rect

widthA = np.sqrt((tl[0] - tr[0])**2 + (tl[1] - tr[1])**2 )
widthB = np.sqrt((bl[0] - br[0])**2 + (bl[1] - br[1])**2 )
maxWidth = max(int(widthA), int(widthB))

heightA = np.sqrt((tl[0] - bl[0])**2 + (tl[1] - bl[1])**2 )
heightB = np.sqrt((tr[0] - br[0])**2 + (tr[1] - br[1])**2 )
maxHeight = max(int(heightA), int(heightB))

### Aplicando a transformação de perspectiva

In [78]:
dst = np.array([[0, 0],
                [maxWidth - 1, 0], 
                [maxWidth - 1, maxHeight - 1], 
                [0, maxHeight - 1]], dtype = 'float32')

transformMatrix = cv2.getPerspectiveTransform(rect, dst)

scan = cv2.warpPerspective(image.copy(), transformMatrix, 
                           (maxWidth, maxHeight))

cv2.imshow('scanneada', scan)
cv2.waitKey(0)
cv2.destroyAllWindows()